In [4]:
!pip install pytorch-forecasting

     |████████████████████████████████| 112 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 810 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 164 kB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 231 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 274 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 14.3 MB/s eta 0:00:01
     |███████████████████████████████

In [1]:
import pandas as pd
import numpy as np
import os
from ts_data import TS_Data
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
from torchvision import transforms, utils

In [2]:
fdir = "/home/warren/Documents/UGRA/AIM/csv_files"

In [3]:
l = os.listdir(fdir)
dir_len = len(l)


In [4]:
l

['UW_VCS_37m_20160831105740.daq.csv',
 'UW_VCS_37m_20160826143151.daq.csv',
 'UW_VCS_37m_20160830095332.daq.csv',
 'UW_VCS_37m_20160927133546.daq.csv',
 'PID104A.daq.csv',
 'UW_VCS_37m_20160919105342.daq.csv',
 'UW_VCS_37m_20160906134747.daq.csv',
 'UW_VCS_37m_20160906133045.daq.csv',
 'UW_VCS_37m_20160707111701.daq.csv',
 'UW_VCS_37m_20160923142631.daq.csv',
 'VCS_PID101B.daq.csv',
 'UW_VCS_37m_20160923121249.daq.csv',
 'UW_VCS_37m_20160920130743.daq.csv',
 'VCS_PID103B.daq.csv',
 'UW_VCS_37m_20160923144110.daq.csv',
 'UW_VCS_37m_20160829104756.daq.csv',
 'UW_VCS_37m_20160826144658.daq.csv',
 'UW_VCS_37m_20161116125417.daq.csv',
 'UW_VCS_37m_20161115114106.daq.csv',
 'UW_VCS_37m_20160923115305.daq.csv',
 'UW_VCS_37m_20160927135151.daq.csv',
 'PID104B.daq.csv',
 'UW_VCS_37m_20161115112515.daq.csv',
 'UW_VCS_37m_20161116124942.daq.csv',
 'minisim_PID112B.daq.csv',
 'UW_VCS_37m_20160830093307.daq.csv',
 'VCS_PID101A.daq.csv',
 'minisim_PID112A.daq.csv',
 'UW_VCS_37m_20160920132052.daq.cs

In [5]:
datasets = [None]*len(l)
for i in range(len(l)):
    temp = os.path.join(fdir,l[i])
    print(temp)
    datasets[i] = TS_Data(temp,8,4)
dataset =  torch.utils.data.ConcatDataset(datasets)
dataloader = DataLoader(dataset, batch_size= 64)
    
    

/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160831105740.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160826143151.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160830095332.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160927133546.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/PID104A.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160919105342.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160906134747.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160906133045.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160707111701.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160923142631.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/VCS_PID101B.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160923121249.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/UW_VCS_37m_20160920130743.daq.csv
/home/warren/Documents/UGRA/AIM/csv_files/VC

In [17]:
dat_cols = ['calculate_lane_deviation_percentage']
for i in range(len(l)):
    temp = os.path.join(fdir,l[i])
    df = pd.read_csv(temp)
    ddf = df[dat_cols].to_numpy()
    print(ddf.shape)
    data = ddf[1:,:] - ddf[:-1,:]
    print(data.shape)
    break 
    

(52906, 3)
(52905, 3)


In [6]:
def get_dataloader(fdir, in_len,pred_len):
    l = os.listdir(fdir)
    datasets = [None]*len(l)
    for i in range(len(l)):
        temp = os.path.join(fdir,l[i])
        datasets[i] = TS_Data(temp, in_len,pred_len)
    dataset =  torch.utils.data.ConcatDataset(datasets)
    dataloader = DataLoader(dataset, batch_size= 64)
    return dataloader


In [7]:
dl = get_dataloader(fdir,8,4)

In [6]:
import pytorch_forecasting

/home/warren/mlenv/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [1]:
import numpy as np

In [5]:
a=np.random.rand(4,3)
print(a)

[[0.9110173  0.47333055 0.85306618]
 [0.17627119 0.47233632 0.00511384]
 [0.23749835 0.95858712 0.10253532]
 [0.94858383 0.25261346 0.78367809]]


In [6]:
a[:,:-1]

array([[0.9110173 , 0.47333055],
       [0.17627119, 0.47233632],
       [0.23749835, 0.95858712],
       [0.94858383, 0.25261346]])

In [7]:
a[:,1:]

array([[0.47333055, 0.85306618],
       [0.47233632, 0.00511384],
       [0.95858712, 0.10253532],
       [0.25261346, 0.78367809]])

In [2]:
from CNN import CNN
import torch
import torch.nn as nn

In [3]:
model = CNN()


In [7]:
sum(p.numel() for p in model.parameters())

62006

In [7]:
model = nn.LSTM(16,40,5) #24 assuming that , check nonlinearities

In [8]:
sum(p.numel() for p in model.parameters())

61760